In [6]:
import pandas as pd
data = pd.read_csv('../dataset_customer_churn.csv', sep='^')
data.drop(labels=['A006_REGISTRO_ANS', 'CODIGO_BENEFICIARIO', 'CLIENTE', 'CD_USUARIO', 'CODIGO_FORMA_PGTO_MENSALIDADE', 'A006_NM_PLANO', 'DIAS_ATE_REALIZAR_ALTO_CUSTO', 'CD_ASSOCIADO', 'ESTADO_CIVIL'], axis=1, inplace=True)
is_NAN = data[data.isna().any(axis=1)]
data.drop(is_NAN.index, axis=0, inplace=True)
dict_replace = {
    "SIM": 1,
    "NAO": 0,
    'F': 0,
    'M': 1,
    'DESATIVADO': 1,
    'ATIVO': 0,

}
data.replace(dict_replace, inplace=True)
data.drop(labels=182212, axis=0, inplace=True)
data_dummified = pd.get_dummies(data)

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

x = data_dummified.drop(labels=['SITUACAO'], axis=1)
y = data_dummified['SITUACAO']
np.random.seed(1010)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, stratify=y)

In [21]:
from sklearn.linear_model import LogisticRegression
log_regression = LogisticRegression(max_iter=20000, penalty='l2', solver='newton-cg')
log_regression.fit(train_x,train_y)
predicts_rfc = log_regression.predict(test_x)

C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\scipy\optimize\_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


In [22]:
from sklearn.metrics import f1_score,recall_score,accuracy_score,precision_score
def all_scores(predicts, test_y):
    print(f"Accuracy Score: {accuracy_score(test_y,predicts):.3f}\nRecall Score: {recall_score(test_y,predicts):.3f}\nPrecision Score: {precision_score(test_y,predicts):.3f}\nF1 Score: {f1_score(test_y,predicts):.3f}\n")
def scores(validation_results):
    media = validation_results['test_score'].mean()
    desvio_padrao = validation_results['test_score'].std()
    print("Accuracy médio {:.2f}".format(media))
    print("Intervalo [{:.2f}, {:.2f}]".format((media - 2 * desvio_padrao), (media + 2 * desvio_padrao)))

In [23]:
all_scores(predicts_rfc, test_y)

Accuracy Score: 0.825
Recall Score: 0.843
Precision Score: 0.837
F1 Score: 0.840



In [19]:
from sklearn.model_selection import cross_validate, KFold

validation_results = cross_validate(log_regression,x,y,cv=KFold(n_splits=10, shuffle=True))
scores(validation_results)
validation_results_df = pd.DataFrame(validation_results)
validation_results_df

C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

Accuracy médio 0.81
Intervalo [0.80, 0.81]


C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,fit_time,score_time,test_score
0,7.591719,0.015958,0.805507
1,6.440529,0.012967,0.799529
2,8.480571,0.018950,0.809881
3,9.003210,0.007979,0.809597
4,6.873283,0.011478,0.809622
5,7.493324,0.016019,0.810088
6,6.949037,0.015957,0.810036
7,7.564354,0.014960,0.809700
8,6.739315,0.017470,0.808380
9,6.864516,0.018949,0.805947


Accuracy médio 0.82
Intervalo [0.82, 0.83]


,fit_time,score_time,test_score
0,171.740070,0.026929,0.823883
1,194.014678,0.011969,0.824866
2,112.726269,0.019946,0.824918
3,145.953824,0.013963,0.824038
4,91.873188,0.013484,0.822278
5,126.729862,0.015957,0.825772
6,171.582896,0.018947,0.824141
7,134.246616,0.018950,0.826082
8,137.171100,0.018949,0.826290
9,181.759161,0.012969,0.824426


In [20]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=log_regression, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(x, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\scipy\optimize\_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Best: 0.824794 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.824794 (0.001349) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.824698 (0.001397) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.824581 (0.001369) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.824792 (0.001346) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.824763 (0.001503) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.824669 (0.001344) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.824752 (0.001344) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.824588 (0.001450) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.824636 (0.001324) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.824688 (0.001381) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.824430 (0.001406) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.824580 (0.001425) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.823312 (0.001370) wit

C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\scipy\optimize\_linesearch.py:415: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Users\ianml\anaconda3\envs\projeto1_WISE\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
